In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
import joblib

In [2]:
df = pd.read_csv("../weatherAUS.csv")
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,12/1/2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,12/2/2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,12/3/2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,12/4/2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,12/5/2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,6/21/2017,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,6/22/2017,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,6/23/2017,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,6/24/2017,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [3]:
#Remove Rows where required columns are null
df1 = df[df['Location'].notna()&df['WindSpeed3pm'].notna()&df['Humidity3pm'].notna()&df['Pressure3pm'].notna()&df['Cloud3pm'].notna()&df['Temp3pm']]

#Remove columns
df1 = df1.drop(['Date','MinTemp','MaxTemp','RainTomorrow'], axis=1)

In [4]:
#Replace with mean values for numerical values
mean_col = ['Rainfall','Evaporation','Sunshine','WindGustSpeed','WindSpeed9am','Humidity9am','Pressure9am','Cloud9am','Temp9am']
df1[mean_col] = df1[mean_col].fillna(df1.mean())

#Replace with mode values for categorical values
mode_col = ['WindGustDir','WindDir9am','WindDir3pm','RainToday']
df1[mode_col] = df1[mode_col].fillna(df1.mode())

C:\Users\HP\AppData\Local\Temp\ipykernel_11940\4114251890.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df1[mean_col] = df1[mean_col].fillna(df1.mean())


In [5]:
#Convert label values into numerical values
le = LabelEncoder()
df1['Location'] = le.fit_transform(df1['Location'])
mapping_Location = dict(zip(le.classes_, range(1, len(le.classes_)+1)))
df1['WindGustDir'] = le.fit_transform(df1['WindGustDir'])
mapping_WindDir = dict(zip(le.classes_, range(1, len(le.classes_)+1)))
df1['WindDir9am'] = le.fit_transform(df1['WindDir9am'])
df1['WindDir3pm'] = le.fit_transform(df1['WindDir3pm'])
df1['RainToday'] = le.fit_transform(df1['RainToday'])
mapping_Rain = dict(zip(le.classes_, range(1, len(le.classes_)+1)))

In [6]:
#Define x,y variables
x = df1.drop(['WindDir3pm','WindSpeed3pm','Humidity3pm','Pressure3pm','Cloud3pm','Temp3pm'], axis=1)
y = df1[['WindDir3pm','WindSpeed3pm','Humidity3pm','Pressure3pm','Cloud3pm','Temp3pm']]

In [7]:
#Divide the dataset for training and testing
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.2)

In [8]:
#Linear Regression
from sklearn.linear_model import LinearRegression  

#Linear regression to the training set model 1
linreg = LinearRegression()  
linreg.fit(x_train, y_train)  

#Predict the test set results model 1
y_pred_test = linreg.predict(x_test)  
x_pred_test = linreg.predict(x_train)  
print('Train Score: {0:0.8f} '.format(linreg.score(x_train, y_train))) 
print('Test Score:  {0:0.8f} '.format(linreg.score(x_test, y_test)))  

Train Score: 0.61012235 
Test Score:  0.61321844 


In [9]:
#Save the trained model
filename = "EveWeather_model.pkl"
joblib.dump(linreg, filename)

['EveWeather_model.pkl']